In [3]:
with open('data.txt', 'r') as f:
    text = f.read()

In [4]:
len(text)

304695

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_abcdefghijklmnopqrstuvwxyz|~¢°äçéíñИОСТШагекмнортуця‍—‘“”•…⏱☕☺♀♂♾⚠❄❓❤⬆녀녕볶붘안애요이자하핫헹힝️🇸🇺🌫🌱🌲🍑🍵🍼🎉🎮🏽🐑🐒👃👋👍👎👏👶👺👿💡💦💨💩💯💰💻🔒🔥🗑🗺😂😃😅😆😇😉😎😓😠😢😤😥😬😭😮😰😶🙂🙃🙄🙏🚌🚗🚪🚶🤣🤮🤷🥂🥳🥹🥺🥾🧙🧠🧩🪵🫖🫠
219


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[71, 72, 72, 1, 83, 71, 68, 81, 68]
hii there


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([304695]) torch.int64
tensor([ 36,  71,  81,  72,  82,   1,  34,  66,  74,  68,  81,  27,   1,  41,
         68,  81,  68,   0,  36,  71,  81,  72,  82,   1,  34,  66,  74,  68,
         81,  27,   1,  58,  78,  84,   1,  77,  64,  76,  68,  67,   1,  83,
         71,  68,   1,  70,  81,  78,  84,  79,   1,  52,  80,  84,  64,  67,
         81,  72,  75,  64,  83,  68,  81,  64,  75,  15,   0,  36,  71,  81,
         72,  82,   1,  34,  66,  74,  68,  81,  27,   1,  58,  78,  84,   1,
         82,  68,  83,   1,  83,  71,  68,   1,  77,  72,  66,  74,  77,  64,
         76,  68,   1,  69,  78,  81,   1,  52,  64,  76,  85,  81,  84,  83,
         71,  64,   1,  53,  84,  76,  84,  75,  84,  81,  84,   1,  83,  78,
          1,  49,  78,  84,  69,  69,  15,   0,  36,  71,  81,  72,  82,   1,
         34,  66,  74,  68,  81,  27,   1,  58,  78,  84,   1,  82,  68,  83,
          1,  88,  78,  84,  81,   1,  77,  72,  66,  74,  77,  64,  76,  68,
          1,  83,  78,   1,  53

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([36, 71, 81, 72, 82,  1, 34, 66, 74])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([36]) the target: 71
when input is tensor([36, 71]) the target: 81
when input is tensor([36, 71, 81]) the target: 72
when input is tensor([36, 71, 81, 72]) the target: 82
when input is tensor([36, 71, 81, 72, 82]) the target: 1
when input is tensor([36, 71, 81, 72, 82,  1]) the target: 34
when input is tensor([36, 71, 81, 72, 82,  1, 34]) the target: 66
when input is tensor([36, 71, 81, 72, 82,  1, 34, 66]) the target: 74


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[78, 81,  1, 83, 71, 68,  1, 69],
        [82,  1, 34, 66, 74, 68, 81, 27],
        [ 1, 56, 68,  1, 76, 84, 82, 83],
        [64, 85, 68,  1, 65, 68, 68, 77]])
targets:
torch.Size([4, 8])
tensor([[81,  1, 83, 71, 68,  1, 69, 72],
        [ 1, 34, 66, 74, 68, 81, 27,  1],
        [56, 68,  1, 76, 84, 82, 83,  8],
        [85, 68,  1, 65, 68, 68, 77,  1]])
----
when input is [78] the target: 81
when input is [78, 81] the target: 1
when input is [78, 81, 1] the target: 83
when input is [78, 81, 1, 83] the target: 71
when input is [78, 81, 1, 83, 71] the target: 68
when input is [78, 81, 1, 83, 71, 68] the target: 1
when input is [78, 81, 1, 83, 71, 68, 1] the target: 69
when input is [78, 81, 1, 83, 71, 68, 1, 69] the target: 72
when input is [82] the target: 1
when input is [82, 1] the target: 34
when input is [82, 1, 34] the target: 66
when input is [82, 1, 34, 66] the target: 74
when input is [82, 1, 34, 66, 74] the target: 68
when input is [82, 1, 3

In [12]:
print(xb), print(yb) # our input to the transformer

tensor([[78, 81,  1, 83, 71, 68,  1, 69],
        [82,  1, 34, 66, 74, 68, 81, 27],
        [ 1, 56, 68,  1, 76, 84, 82, 83],
        [64, 85, 68,  1, 65, 68, 68, 77]])
tensor([[81,  1, 83, 71, 68,  1, 69, 72],
        [ 1, 34, 66, 74, 68, 81, 27,  1],
        [56, 68,  1, 76, 84, 82, 83,  8],
        [85, 68,  1, 65, 68, 68, 77,  1]])


(None, None)

In [13]:
torch.randint(len(data) - block_size, (batch_size,))

tensor([171255, 134768, 207377, 265624])

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        self.lin = nn.Linear(vocab_size, vocab_size)
        

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        tet = self.token_embedding_table(idx) # (B,T,C)
        logits = self.lin(tet)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(logits)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=20)[0].tolist()))


torch.Size([32, 219])
tensor([[-0.5245, -0.4027, -0.0303,  ..., -0.9174,  0.9790,  0.6354],
        [ 0.7274,  0.7782, -0.6340,  ...,  0.3387,  0.3893,  0.0223],
        [ 1.1733,  0.2839, -0.2384,  ...,  0.0989, -0.8305, -0.0563],
        ...,
        [-0.7373, -0.2838, -0.4931,  ..., -0.1127,  0.0679,  0.0794],
        [-0.7373, -0.2838, -0.4931,  ..., -0.1127,  0.0679,  0.0794],
        [ 0.6262, -0.7645,  0.5870,  ...,  0.1103,  0.1233,  0.4153]],
       grad_fn=<ViewBackward0>)
tensor(5.4546, grad_fn=<NllLossBackward0>)

🤮j4'*l👋🎮?🔥🚌4😢😆🌫😎A⚠이😶


In [15]:
vocab_size

219

In [16]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-2)

In [20]:
batch_size = 32
block_size=7
for steps in range(3000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.37160325050354


In [26]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Dapluth man Strinee phacosto. r ledevrtorutrumvtho
Zamusthat flyiere s
Strizhrug'mvrkedespizhenie I'


In [19]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Cha sthit'sevried Sand/osende?
Strievelife The witinvravs wwevrid a act PSated thache: t 🔒
Sar: my w


In [27]:
torch.cuda.is_available()

True

In [ ]:
print(torch.version)

In [28]:
batch_size

32

In [29]:
testmodel = torch.load('gpt_model.pt')

AttributeError: Can't get attribute 'GPTLanguageModel' on <module '__main__'>